# Person Detection

In [ ]:
import cv2

# Load the pre-trained Haar Cascade Classifier for pedestrian detection
person_cascade_path = "haarcascade_fullbody.xml"
person_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + person_cascade_path)

# Open a video capture stream (you can also use a webcam or video file)
cap = cv2.VideoCapture("person.mp4")

# Get the frames per second (fps) of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter("person_output.avi", fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale (for faster processing)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect pedestrians in the frame
    pedestrians = person_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw bounding boxes around detected pedestrians and label them as "person"
    for (x, y, w, h) in pedestrians:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, "person", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Write the frame with bounding boxes to the output video
    output.write(frame)

    # Display the frame with bounding boxes
    cv2.imshow("Tunnel Detection", frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and video writer, and close OpenCV windows
cap.release()
output.release()
cv2.destroyAllWindows()


# Car Detection

In [ ]:
import cv2
import numpy as np

# Load YOLOv4 model
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")

# Load COCO class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Open a video capture stream (you can also use a webcam or video file)
cap = cv2.VideoCapture("car.mp4")

# Get the frame dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object to save the output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter("car_output.avi", fourcc, 20.0, (width, height))

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for YOLOv4
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Get the names of the output layers
    output_layers = net.getUnconnectedOutLayersNames()

    # Run YOLO object detection
    detections = net.forward(output_layers)

    # Process detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5 and class_id in [2, 5, 7]:  # Class IDs for car, bus, and truck in COCO dataset
                # Object detected with confidence > 0.5 and belongs to car, bus, or truck classes
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)

                # Calculate the top-left corner of the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Assign different colors based on the class
                if class_id == 2:  # Car
                    color = (0, 255, 0)  # Green
                elif class_id == 5:  # Bus
                    color = (255, 0, 0)  # Blue
                elif class_id == 7:  # Truck
                    color = (0, 0, 255)  # Red

                # Draw bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

                # Display class name above the bounding box
                label = f"{classes[class_id]}"
                y_label = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, label, (x, y_label), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the frame with bounding boxes to the output video
    output.write(frame)

    # Display the frame with bounding boxes
    cv2.imshow("Object Detection", frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture, video writer, and close OpenCV windows
cap.release()
output.release()
cv2.destroyAllWindows()


# Fire Detection

In [7]:
import cv2
import numpy as np

# Open a video capture stream (you can also use a webcam or video file)
cap = cv2.VideoCapture("fire_tunnel.mp4")

# Get the frame dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object to save the output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter("output_fire_detection_alternate.avi", fourcc, 20.0, (width, height))

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define lower and upper bounds for fire color in HSV
    lower_bound = np.array([0, 120, 50])
    upper_bound = np.array([10, 255, 255])

    # Create a binary mask by thresholding based on color
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # Apply morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Process detected contours
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 100:  # Adjust the area threshold as needed
            # Draw bounding box
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

            # Display "fire" label above the bounding box
            label = "fire"
            y_label = y - 15 if y - 15 > 15 else y + 15
            cv2.putText(frame, label, (x, y_label), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Write the frame with bounding boxes to the output video
    output.write(frame)

    # Display the frame with bounding boxes
    cv2.imshow("Fire Detection", frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture, video writer, and close OpenCV windows
cap.release()
output.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load YOLOv3 model for general object detection
net_objects = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
classes_objects = []
with open("coco.names", "r") as f:
    classes_objects = [line.strip() for line in f.readlines()]

# Open the camera stream (use 0 for the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects using YOLOv3 for general object detection
    blob_objects = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net_objects.setInput(blob_objects)
    output_objects = net_objects.forward(net_objects.getUnconnectedOutLayersNames())

    # Process object detections
    for detection in output_objects:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Get bounding box coordinates
                box = obj[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype("int")

                # Draw bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Display label above the bounding box
                label = f"{classes_objects[class_id]}: {confidence:.2f}"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with object detections
    cv2.imshow("Object Detection", frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
